In [1]:
import os
import sys
import importlib
if '../utils' not in sys.path:
    sys.path.insert(0,'../utils')
import lmp
import lmpsetup
import analysis
import numpy as np

importlib.reload(lmp);
importlib.reload(lmpsetup);
importlib.reload(analysis);

### CPEB4 setup
We setup two independent replica exchange simulations for CPEB4, ranging from 150K to 600K :

In [2]:
cpeb4 = 'MGDYGFGVLVQSNTGNKSAFPVRFHPHLQPPHHHQNATPSPAAFINNNTAANGSSAGSAWLFPAPATHNIQDEILGSEKAKSQQQEQQDPLEKQQLSPSPGQEAGILPETEKAKSEENQGDNSSENGNGKEKIRIESPVLTGFDYQEATGLGTSTQPLTSSASSLTGFSNWSAAIAPSSSTIINEDASFFHQGGVPAASANNGALLFQNFPHHVSPGFGGSFSPQIGPLSQHHPHHPHFQHHHSQHQQQRRSPASPHPPPFTHRNAAFNQLPHLANNLNKPPSPWSSYQSPSPTPSSSWSPGGGGYGGWGGSQGRDHRRGLNGGITPLNSISPLKKNFASNHIQLQKYARPSSAFAPKSWMEDSLNRADNIFPFPDRPRTFDMHSLESSLIDIMRAENDTIKARTYGRRRGQSSLFPMEDGFLDDGRGDQPLHSGLGSPHCFSHQNGE'

In [3]:
dir1 = '/home/adria/perdiux/prod/lammps/dignon/CPEB4/RE-cpeb4'
c1 = lmpsetup.LMPSetup(oliba_wd=dir1, seq=cpeb4, temper=True)
c1.temperatures = '150.0 170.0 192.5 217.5 247.5 280.0 320.0 362.5 410.0 467.5 530.0 600.0'
c1.processors = 12
c1.t = 100000000
c1.get_equilibration_xyz()
c1.get_hps_params()
c1.get_hps_pairs()
c1.write_hps_files()

In [4]:
dir2 = '/home/adria/perdiux/prod/lammps/dignon/CPEB4/2-RE-cpeb4'
c2 = lmpsetup.LMPSetup(oliba_wd=dir2, seq=cpeb4, temper=True)
c2.temperatures = '150.0 170.0 192.5 217.5 247.5 280.0 320.0 362.5 410.0 467.5 530.0 600.0'
c2.processors = 12
c2.t = 100000000
c2.get_equilibration_xyz()
c2.v_seed = c1.v_seed + 1
c2.langevin_seed = c1.langevin_seed + 1
c2.get_hps_params()
c2.get_hps_pairs()
c2.write_hps_files()

We also wish to setup dynamics with a varying scale on lambda

In [7]:
rang = np.linspace(0.55, 1, 4)
l_base = lmpsetup.LMPSetup(oliba_wd='', seq=cpeb4, temper=True)
l_base.processors = 12
l_base.t = 100000000
l_base.get_equilibration_xyz()
for r in rang:
    d = f'/home/adria/perdiux/prod/lammps/dignon/CPEB4/{r:.2f}-lS-cpeb4'
    os.mkdir(d)
    l_base.o_wd = d
    l_base.hps_scale = r
    l_base.get_hps_params()
    l_base.get_hps_pairs()
    l_base.write_hps_files()